<a href="https://colab.research.google.com/github/shaankhurshid/bmi707/blob/pre-processing/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [1]:
# Copying the data from dropbox
!wget https://www.dropbox.com/s/neharvmmj5wkftk/train_controls_transformed.csv?dl=0
!wget https://www.dropbox.com/s/l7ttj1jjd7rtwbp/train_cases_transformed.csv?dl=0
!wget https://www.dropbox.com/s/f4zsvv2eeia5lbl/holdout_controls_transformed.csv?dl=0
!wget https://www.dropbox.com/s/eqpxj42chflzfl4/holdout_cases_transformed.csv?dl=0

--2020-04-20 16:32:01--  https://www.dropbox.com/s/neharvmmj5wkftk/train_controls_transformed.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/neharvmmj5wkftk/train_controls_transformed.csv [following]
--2020-04-20 16:32:01--  https://www.dropbox.com/s/raw/neharvmmj5wkftk/train_controls_transformed.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2a72bc4998011d43f6b9c118a5.dl.dropboxusercontent.com/cd/0/inline/A2MuepXO3kWTCE1RNO5QlJmU7ZHHY1Ny7UBtDfOYpYmfwlbLv5e0xJ45cvx7ZyWmBNO1Qxd6msiNmLtcWO5FEDkg_GB5y3w9r7NnkEIO39eMufBjcHr4pbqgeUplBaFIf6Y/file# [following]
--2020-04-20 16:32:02--  https://uc2a72bc4998011d43f6b9c118a5.dl.dropboxusercontent.com/cd/0/inline/A2MuepXO3kWTCE1RNO5QlJmU7ZHHY1Ny7UBtDfOYpYmfwlbLv5e0xJ4

NameError: ignored

In [0]:
# Importing the data
train_controls = pd.read_csv('train_controls_transformed.csv?dl=0')
train_cases = pd.read_csv('train_cases_transformed.csv?dl=0')
train_controls = pd.read_csv('holdout_controls_transformed.csv?dl=0')
train_cases = pd.read_csv('holdout_cases_transformed.csv?dl=0')